In [29]:
from matplotlib import pyplot as plt
import numpy as np
import pandas as pd
from rdkit import Chem
from rdkit import DataStructs
from rdkit.Chem import AllChem, Draw, Lipinski, Descriptors
from rdkit.ML.Descriptors import MoleculeDescriptors
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import mean_squared_error, r2_score

In [30]:
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch_geometric.data import Data, Batch
from torch_geometric.nn import GCNConv, GATConv
from torch.optim.lr_scheduler import StepLR
from torch.utils.data import DataLoader

In [75]:
df = pd.read_csv('extra_data.csv')
id_list = df['ID'].values
extra_data = df.drop(columns=['ID']).values # 删除id列，提取特征数据
for item in extra_data:
    print(item)
scaler = StandardScaler()
extra_data_scale = scaler.fit_transform(extra_data)
extra_data_with_id = pd.DataFrame(extra_data_scale, columns=df.columns[1:])
extra_data_with_id.insert(0, 'ID', id_list) # 将id插入到第一列位置

[1.00e+01 3.13e+02 2.00e-02]
[5.00e+01 3.13e+02 2.00e-02]
[1.00e+02 3.13e+02 2.00e-02]
[1.50e+02 3.13e+02 2.00e-02]
[2.00e+02 3.13e+02 2.00e-02]
[2.587e+00 3.330e+02 1.000e-01]
[2.453000e+01 2.980000e+02 9.644383e-02]
[2.4530000e+01 2.9800000e+02 2.0666535e-01]
[ 24.53       298.           0.48221915]
[ 24.53      298.          0.9644383]
[2.4530000e+01 2.9800000e+02 1.0037139e-01]
[2.4530000e+01 2.9800000e+02 2.1508155e-01]
[ 24.53       298.           0.50185695]
[ 24.53      298.          1.0037139]
[3.50000e+01 3.23000e+02 2.65272e-03]
[3.5000e+01 3.2300e+02 6.6318e-03]
[3.50000e+01 3.23000e+02 1.32636e-02]
[3.50000e+01 3.23000e+02 2.65272e-02]
[3.50000e+01 3.23000e+02 1.32636e-01]
[3.5000e+01 3.2300e+02 3.4137e-03]
[3.50000e+01 3.23000e+02 8.53425e-03]
[3.50000e+01 3.23000e+02 1.70685e-02]
[3.5000e+01 3.2300e+02 3.4137e-02]
[3.50000e+01 3.23000e+02 1.70685e-01]
[3.5000e+01 3.2300e+02 3.4137e-03]
[3.50000e+01 3.23000e+02 8.53425e-03]
[3.50000e+01 3.23000e+02 1.70685e-02]
[3.5000e+0

In [56]:
extra_data_with_id

,ID,NaClg/L,TempK,Concentrationg/L
0,10001.0,-0.825422,-0.302082,-0.039856
1,10002.0,0.553632,-0.302082,-0.039856
2,10003.0,2.277450,-0.302082,-0.039856
3,10004.0,4.001268,-0.302082,-0.039856
4,10005.0,5.725086,-0.302082,-0.039856
...,...,...,...,...
1677,11718.0,0.036487,-1.069721,-0.039855
1678,11719.0,0.036487,-1.069721,-0.039854
1679,11720.0,0.036487,-0.302082,-0.039855
1680,11721.0,0.036487,0.209676,-0.039855


In [45]:
data = pd.read_csv('molecules.csv', encoding='ANSI')
molecules = []
max_atoms = 158 #最大原子数
max_edges = 0 # 最大边数
#按照id分组处理
for  _,group in data.groupby('ID'):
    for _,row in group.iterrows():
        IE = row['IE']
        smiles = row['SMILES']
        molecule_id = row['ID']
      #  print(smiles)
        mol = Chem.MolFromSmiles(smiles)
        # SMILE 无效 → 跳过
        if mol is None:
            continue
        num_atoms = len(mol.GetAtoms()) # 原子数获取
        max_atoms = max(max_atoms, num_atoms) # 最大原子数更新
        bonds = mol.GetBonds()
        num_edges = len(bonds) #边数获取
#             adj_matrix = np.zeros((num_atoms, num_atoms)) # 邻接矩阵初始化
#             # 填充邻接矩阵，建立连接关系(无向图)
#             for bond in bonds:
#                 i = bond.GetBeginAtomIdx()
#                 j = bond.GetEndAtomIdx()
#                 adj_matrix[i,j] = 1
#                 adj_matrix[j,i] = 1 #无向图
        adj_matrix = Chem.GetAdjacencyMatrix(mol)  # 使用RDKit直接获取邻接矩阵
        # 将原子特征转换为(num_atoms, 148)形状
        atom_features = []
        for atom in mol.GetAtoms():
            features = [
                atom.GetAtomicNum(),
                atom.GetDegree(),
                atom.GetFormalCharge(),  # 原子的形式电荷
                int(atom.HasProp('_ChiralityPossible')),  # 手性信息
                atom.GetTotalNumHs(),  # 连接的H原子数
                atom.GetHybridization().real,  # 杂化类型
                atom.GetExplicitValence(),  # 显式化合价
                int(atom.GetIsAromatic()),  # 芳香性
            ]
            atom_features.append(features)
        features = np.array(atom_features, dtype=np.float32) #强制转换为float32
        atom_features = torch.tensor(features, dtype=torch.float) # (num_atoms, 148)
        # 填充atom_features 到 max_atom
        if num_atoms < max_atoms:
            padding = torch.zeros((max_atoms-num_atoms, 8), dtype=torch.float)
            atom_features = torch.cat([atom_features, padding], dim=0)
        # 处理邻接矩阵，转化为edge_index
        edge_index = torch.tensor(np.array(np.nonzero(adj_matrix)), dtype=torch.long) # (2, num_edges)
        # 更新最大边数的计算，基于实际边数
        max_edges = max(max_edges, edge_index.shape[1])
#             # 如果edge_index边数小于max_edges,需要填充到max_edges
#             if edge_index.shape[i] < max_edges:
#                 padding_edges = torch.zeros((2, int(max_edges - edge_index.shipe[i])), dtype=torch.long)
#                 edge_index = torch.cat([edge_index, padding_edges], dim=1)

        #print(f"edge_index.shape:{edge_index.shape}")
        #print(f"atom_feature.shape:{atom_features.shape}")

        #保存分子信息
        molecules.append ({
            'x':atom_features,
            'edge_index': edge_index,
            'smiles': smiles,
            'id': molecule_id,
            'y':IE
        })
molecules

7 158
7 158
7 158
7 158
7 158
17 158
91 158
91 158
91 158
91 158
95 158
95 158
95 158
95 158
20 158
20 158
20 158
20 158
20 158
26 158
26 158
26 158
26 158
26 158
26 158
26 158
26 158
26 158
26 158
26 158
26 158
26 158
26 158
26 158
46 158
46 158
46 158
46 158
46 158
46 158
46 158
46 158
46 158
46 158
46 158
46 158
46 158
29 158
29 158
29 158
29 158
29 158
29 158
29 158
29 158
29 158
29 158
29 158
29 158
29 158
29 158
29 158
7 158
7 158
9 158
9 158
10 158
10 158
10 158
10 158
10 158
10 158
10 158
53 158
53 158
53 158
53 158
53 158
53 158
18 158
4 158
22 158
25 158
8 158
32 158
32 158
32 158
32 158
33 158
33 158
33 158
33 158
7 158
7 158
7 158
7 158
7 158
14 158
14 158
14 158
14 158
14 158
14 158
14 158
14 158
14 158
13 158
13 158
13 158
13 158
42 158
42 158
42 158
42 158
42 158
22 158
22 158
22 158
22 158
22 158
22 158
10 158
10 158
8 158
8 158
20 158
20 158
20 158
20 158
20 158
20 158
34 158
34 158
34 158
34 158
34 158
34 158
35 158
35 158
35 158
35 158
35 158
35 158
11 158
11 158
11 

10 158
10 158
10 158
10 158
10 158
10 158
10 158
25 158
20 158
25 158
30 158
30 158
30 158
30 158
30 158
30 158
30 158
30 158
30 158
30 158
30 158
30 158
30 158
30 158
30 158
30 158
30 158
30 158
30 158
30 158
28 158
28 158
28 158
28 158
28 158
28 158
28 158
33 158
33 158
33 158
39 158
39 158
25 158
25 158
25 158
25 158
25 158
25 158
25 158
25 158
25 158
25 158
25 158
25 158
20 158
20 158
20 158
20 158
20 158
20 158
25 158
25 158
44 158
13 158
13 158
13 158
13 158
13 158
13 158
25 158
25 158
44 158
25 158
25 158
44 158
25 158
25 158
25 158
25 158
25 158
25 158
25 158
15 158
15 158
15 158
15 158
30 158
30 158
30 158
30 158
30 158
30 158
30 158
30 158
30 158
30 158
30 158
29 158
29 158
29 158
29 158
29 158
29 158
29 158
29 158
29 158
29 158
6 158
6 158
6 158
6 158
6 158
6 158
6 158
6 158
6 158
6 158
12 158
12 158
12 158
12 158
12 158
8 158
8 158
8 158
8 158
8 158
10 158
10 158
10 158
10 158
10 158
6 158
6 158
6 158
6 158
6 158
6 158
6 158
6 158
6 158
6 158
12 158
12 158
12 158
12 158
12 

[{'x': tensor([[16.,  1.,  0.,  ...,  3.,  2.,  0.],
          [ 6.,  3.,  0.,  ...,  3.,  4.,  1.],
          [ 7.,  2.,  0.,  ...,  3.,  3.,  1.],
          ...,
          [ 0.,  0.,  0.,  ...,  0.,  0.,  0.],
          [ 0.,  0.,  0.,  ...,  0.,  0.,  0.],
          [ 0.,  0.,  0.,  ...,  0.,  0.,  0.]]),
  'edge_index': tensor([[0, 1, 1, 1, 2, 2, 3, 3, 4, 4, 5, 5, 6, 6],
          [1, 0, 2, 6, 1, 3, 2, 4, 3, 5, 4, 6, 1, 5]]),
  'smiles': 'S=c1nccc[nH]1',
  'id': 10001.0,
  'y': 0.9124},
 {'x': tensor([[16.,  1.,  0.,  ...,  3.,  2.,  0.],
          [ 6.,  3.,  0.,  ...,  3.,  4.,  1.],
          [ 7.,  2.,  0.,  ...,  3.,  3.,  1.],
          ...,
          [ 0.,  0.,  0.,  ...,  0.,  0.,  0.],
          [ 0.,  0.,  0.,  ...,  0.,  0.,  0.],
          [ 0.,  0.,  0.,  ...,  0.,  0.,  0.]]),
  'edge_index': tensor([[0, 1, 1, 1, 2, 2, 3, 3, 4, 4, 5, 5, 6, 6],
          [1, 0, 2, 6, 1, 3, 2, 4, 3, 5, 4, 6, 1, 5]]),
  'smiles': 'S=c1nccc[nH]1',
  'id': 10002.0,
  'y': 0.8556},
 {'x': t

In [44]:
max_atoms

158

In [11]:
class GraphNN(nn.Module):
    def __init__(self, input_dim, output_dim, hidden_dim=64, dropout=0.3, num_layers=2):
        super().__init__()
        #定义图神经网络编码器层
        self.encoder = nn.ModuleList([
            GATConv(input_dim if i == 0 else hidden_dim, hidden_dim)
            for i in range(num_layers)
        ])
        #定义解码器层
        self.decoder = GATConv(hidden_dim, output_dim)
        #Dropout层
        self.dropout = dropout
        #归一化层
        self.layer_norm = nn.LayerNorm(hidden_dim)
        #初始化网络参数
        self._initialize_weights()
        
    def _initialize_weights(self):
        # 对图卷积层权重初始化
        for layer in self.encoder:
            if isinstance(layer, GATConv):
                self.initialize_gatconv_weights(layer)

        # 解码器层初始化
        if isinstance(self.decoder, GATConv):
            self.initialize_gatconv_weights(self.decoder)
            
    def initialize_gatconv_weights(self, gat_layer):
        # 确保 gat_layer 不是 None
        if gat_layer.lin_src is not None and gat_layer.lin_dst is not None:
            nn.init.xavier_uniform_(gat_layer.lin_src.weight)
            nn.init.xavier_uniform_(gat_layer.lin_dst.weight)
            if gat_layer.attn_vecs is not None:
                nn.init.xavier_uniform_(gat_layer.attn_vecs)

    def forward(self, batch):
        x, edge_index = batch.x, batch.edge_index
        edge_attr = batch.edge_attr if 'edge_attr' in batch else None

        #前向传播过程，编码器+激活函数+层归一化
        for layer in self.encoder:
            x = F.dropout(x, p=self.dropout, training=self.training)
            x = layer(x, edge_index, edge_attr=edge_attr)
            x = F.relu(x)
            x = self.layer_norm(x) #归一化

        #解码层输出
        output = self.decoder(x, edge_index, edge_attr=edge_attr)
        return output

In [10]:
molecules[0],molecules[10]

({'x': tensor([[16.,  1.,  0.,  0.,  0.,  3.,  2.,  0.],
          [ 6.,  3.,  0.,  0.,  0.,  3.,  4.,  1.],
          [ 7.,  2.,  0.,  0.,  0.,  3.,  3.,  1.],
          [ 6.,  2.,  0.,  0.,  1.,  3.,  3.,  1.],
          [ 6.,  2.,  0.,  0.,  1.,  3.,  3.,  1.],
          [ 6.,  2.,  0.,  0.,  1.,  3.,  3.,  1.],
          [ 7.,  2.,  0.,  0.,  1.,  3.,  3.,  1.]]),
  'edge_index': tensor([[0, 1, 1, 1, 2, 2, 3, 3, 4, 4, 5, 5, 6, 6],
          [1, 0, 2, 6, 1, 3, 2, 4, 3, 5, 4, 6, 1, 5]]),
  'smiles': 'S=c1nccc[nH]1',
  'id': 10001.0},
 {'x': tensor([[ 6.,  1.,  0.,  0.,  3.,  4.,  1.,  0.],
          [ 6.,  2.,  0.,  0.,  2.,  4.,  2.,  0.],
          [ 6.,  2.,  0.,  0.,  2.,  4.,  2.,  0.],
          [ 6.,  2.,  0.,  0.,  2.,  4.,  2.,  0.],
          [ 6.,  2.,  0.,  0.,  2.,  4.,  2.,  0.],
          [ 6.,  2.,  0.,  0.,  2.,  4.,  2.,  0.],
          [ 6.,  2.,  0.,  0.,  2.,  4.,  2.,  0.],
          [ 6.,  2.,  0.,  0.,  2.,  4.,  2.,  0.],
          [ 6.,  3.,  0.,  1.,  1.,  

In [14]:
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch_geometric.data import Data, Batch
from torch_geometric.nn import GATConv, global_mean_pool
from torch.optim.lr_scheduler import StepLR
from torch.utils.data import DataLoader
from matplotlib import pyplot as plt
import numpy as np
import pandas as pd
from rdkit import Chem
from rdkit import DataStructs
from rdkit.Chem import AllChem, Draw, Lipinski, Descriptors
from rdkit.ML.Descriptors import MoleculeDescriptors
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler

def load_data(csv_file):
    data = pd.read_csv(csv_file, encoding='ANSI')
    molecules = []
    max_atoms = 158 #最大原子数
    max_edges = 0 # 最大边数
    for _, group in data.groupby('ID'):
        for _, row in group.iterrows():
            IE = row['IE']
            smiles = row['SMILES']
            molecule_id = row['ID']
            mol = Chem.MolFromSmiles(smiles)
            if mol is None:
                continue
            num_atoms = len(mol.GetAtoms())
            max_atoms = max(max_atoms, num_atoms)
            bonds = mol.GetBonds()
            num_edges = len(bonds)
            adj_matrix = Chem.GetAdjacencyMatrix(mol)
            atom_features = []
            for atom in mol.GetAtoms():
                features = [
                    atom.GetAtomicNum(),
                    atom.GetDegree(),
                    atom.GetFormalCharge(),
                    int(atom.HasProp('_ChiralityPossible')),
                    atom.GetTotalNumHs(),
                    atom.GetHybridization().real,
                    atom.GetExplicitValence(),
                    int(atom.GetIsAromatic()),
                ]
                atom_features.append(features)
            features = np.array(atom_features, dtype=np.float32)
            atom_features = torch.tensor(features, dtype=torch.float)
            if num_atoms < max_atoms:
                padding = torch.zeros((max_atoms - num_atoms, 8), dtype=torch.float)
                atom_features = torch.cat([atom_features, padding], dim=0)
            edge_index = torch.tensor(np.array(np.nonzero(adj_matrix)), dtype=torch.long)
            max_edges = max(max_edges, edge_index.shape[1])
            molecules.append({
                'x': atom_features,
                'edge_index': edge_index,
                'smiles': smiles,
                'id': molecule_id,
                'y': torch.tensor([IE], dtype=torch.float)
            })
    return molecules

def load_extra_data(csv_file):
    df = pd.read_csv(csv_file)
    df = df.drop(columns=['IE','SMILES'])
    id_list = df['ID'].values
    extra_data = df.drop(columns=['ID']).values
    scaler = StandardScaler()
    extra_data_scale = scaler.fit_transform(extra_data)
    extra_data_with_id = pd.DataFrame(extra_data_scale, columns=df.columns[1:])
    extra_data_with_id.insert(0, 'ID', id_list)
    return id_list, extra_data_with_id

class GraphNN(nn.Module):
    def __init__(self, input_dim, output_dim, hidden_dim=64, dropout=0.3, num_layers=2):
        super().__init__()
        self.encoder = nn.ModuleList([
            GATConv(input_dim if i == 0 else hidden_dim, hidden_dim)
            for i in range(num_layers)
        ])
        self.decoder = GATConv(hidden_dim, output_dim)
        self.dropout = dropout
        self.layer_norm = nn.LayerNorm(hidden_dim)
        self._initialize_weights()

    def _initialize_weights(self):
        for layer in self.encoder:
            if isinstance(layer, GATConv):
                self.initialize_gatconv_weights(layer)

        if isinstance(self.decoder, GATConv):
            self.initialize_gatconv_weights(self.decoder)

    def initialize_gatconv_weights(self, gat_layer):
        if gat_layer.lin_src is not None and gat_layer.lin_dst is not None:
            nn.init.xavier_uniform_(gat_layer.lin_src.weight)
            nn.init.xavier_uniform_(gat_layer.lin_dst.weight)
            if gat_layer.attn_vecs is not None:
                nn.init.xavier_uniform_(gat_layer.attn_vecs)

    def forward(self, batch):
        x, edge_index = batch.x, batch.edge_index
        for layer in self.encoder:
            x = F.dropout(x, p=self.dropout, training=self.training)
            x = layer(x, edge_index)
            x = F.relu(x)
            x = self.layer_norm(x)
        output = self.decoder(x, edge_index)
        graph_embeddings = global_mean_pool(output, batch.batch) 
        return graph_embeddings

def train_graph_nn(model, data_loader, optimizer, clip_val=1.0):
    model.train()
    total_loss = 0
    loss_fun = torch.nn.MSELoss()
    num_samples = 0
    for data in data_loader:
        optimizer.zero_grad()
        length = data.x.size(0)
        output = model(data.to(torch.device('cuda' if torch.cuda.is_available() else 'cpu')))
        #y = data.y.to(torch.device('cuda' if torch.cuda.is_available() else 'cpu'))
        #y = torch.zeros((length,1)).to(torch.device('cuda' if torch.cuda.is_available() else 'cpu'))
        y = data.y.to(torch.device('cuda' if torch.cuda.is_available() else 'cpu')).view(-1)
            
        # 检查形状是否匹配
        if output.size(0) != y.size(0):
            raise ValueError(f"Shape mismatch: output {output.shape}, y {y.shape}")
#         print(y.shape)
#         print(output.shape)
        loss = loss_fun(output.squeeze(), y)
        loss.backward()
        torch.nn.utils.clip_grad_norm_(model.parameters(), clip_val)
        optimizer.step()
        total_loss += loss.item() *  y.size(0)
        num_samples +=  y.size(0)
    avg_loss = total_loss / num_samples if num_samples > 0 else float('nan')
    return avg_loss

def collate_fn(batch):
    x_list = [data['x'] for data in batch]
    edge_index_list = [data['edge_index'] for data in batch]
    y_list = [data['y'] for data in batch]

    x = torch.cat(x_list, dim=0)
    edge_index = []
    num_nodes_accum = 0
    for i, edge_idx in enumerate(edge_index_list):
        num_nodes = x_list[i].size(0)  # 使用x的节点数而非边索引最大值
        edge_idx_offset = edge_idx + num_nodes_accum
        edge_index.append(edge_idx_offset)
        num_nodes_accum += num_nodes  # 累加实际节点数

    edge_index = torch.cat(edge_index, dim=1)
    y = torch.cat(y_list, dim=0)
    
    # Create batch tensor
    batch_indices = []
    for i, data in enumerate(batch):
        num_nodes = data['x'].size(0)
        batch_indices.append(torch.full((num_nodes,), i, dtype=torch.long))
    batch_tensor = torch.cat(batch_indices, dim=0)


    batch_data = Batch(x=x, edge_index=edge_index, y=y, batch=batch_tensor)
    return batch_data

class Fun_Predictor(nn.Module):
    def __init__(self, input_size, hidden_size, dropout_rate=0.5):
        super(Fun_Predictor, self).__init__()
        self.fc1 = nn.Linear(input_size, hidden_size)
       # self.bn1 = nn.BatchNorm1d(hidden_size)
        self.bn1 = nn.LayerNorm(hidden_size)
        self.dropout = nn.Dropout(dropout_rate)
        self.fc2 = nn.Linear(hidden_size, hidden_size)
        #self.bn2 = nn.BatchNorm1d(hidden_size)
        self.bn2 = nn.LayerNorm(hidden_size)
        self.fc3 = nn.Linear(hidden_size, hidden_size)
        self.fc4 = nn.Linear(hidden_size, 1)
        self._initialize_weights()

    def _initialize_weights(self):
        for m in self.modules():
            if isinstance(m, nn.Linear):
                nn.init.xavier_uniform_(m.weight)
                if m.bias is not None:
                    nn.init.constant_(m.bias, 0)

    def forward(self, combined_features):
        x = torch.relu(self.fc1(combined_features))
        x = self.bn1(x)
        x = self.dropout(x)
        x = torch.relu(self.fc2(x))
        x = self.bn2(x)
        x = self.dropout(x)
        x = torch.relu(self.fc3(x))
        predict_IE = self.fc4(x)
        return predict_IE

def train_fun_predictor(model_fun, graph_nn, molecule_data, extra_data_with_id, criterion, optimizer_fun):
    model_fun.train()
    graph_nn.eval()
    total_loss = 0
    
    # 创建一个字典，将额外特征与id匹配，去除id列
    extra_data_dict = {row['ID']: torch.tensor(row[1:], dtype=torch.float32) for _, row in extra_data_with_id.iterrows()}
    
    # 初始化一个列表保存额外特征值
    extra_features = []
    targets = []
    graph_embeddings = []

    # 遍历所有额外特征数据，并为每个分子获取对应的图嵌入和特征
    for _, row in extra_data_with_id.iterrows():
        id_ = row['ID']
        extra_feature_tensor = extra_data_dict.get(id_)
        molecule_data_for_id = next((data for data in molecule_data if data['id'] == id_), None)
        if molecule_data_for_id is None:
            raise ValueError(f"No mol data found for ID: {id_}")
        x = molecule_data_for_id['x'] # 原子特征
        edge_index = molecule_data_for_id['edge_index'] # 邻接矩阵
        graph_data = Data(x=x, edge_index=edge_index)
        
        # 获取图嵌入
        with torch.no_grad():
            embedding = graph_nn(graph_data.to(torch.device('cuda' if torch.cuda.is_available() else 'cpu')))
            pooled_embedding = torch.mean(embedding, dim=0).unsqueeze(0) #平均池化
        
        # 拼接图嵌入和额外特征
        combined_features = torch.cat((pooled_embedding, extra_feature_tensor.unsqueeze(0)), dim=1)
        extra_features.append(combined_features)
        targets.append(molecule_data_for_id['y'])
        graph_embeddings.append(pooled_embedding)

    # 将所有特征和目标转换为张量
    extra_features = torch.cat(extra_features, dim=0)
    targets = torch.cat(targets, dim=0)
    graph_embeddings = torch.cat(graph_embeddings, dim=0)

    # 训练循环
    for i in range(len(extra_features)):
        combined_feature = extra_features[i].unsqueeze(0)
        target = targets[i].unsqueeze(0)
        optimizer_fun.zero_grad()
        prediction = model_fun(combined_feature.to(torch.device('cuda' if torch.cuda.is_available() else 'cpu')))
        loss = criterion(prediction.squeeze(), target.to(torch.device('cuda' if torch.cuda.is_available() else 'cpu')))
        loss.backward()
        optimizer_fun.step()
        total_loss += loss.item()
    
    avg_loss = total_loss / len(extra_data_with_id)
    return avg_loss

# 示例使用
if __name__ == "__main__":
    csv_file_molecules = 'molecules.csv'
    csv_file_extra = 'extra_data.csv'
    csv_file = 'solution_properties_converted.csv'

    molecule_data = load_data(csv_file)
    id_list, extra_data_with_id = load_extra_data(csv_file)

    input_dim = 8  # 原子特征维度
    output_dim = 1  # 图嵌入输出维度
    hidden_dim = 64
    dropout = 0.3
    num_layers = 2

    graph_nn = GraphNN(input_dim, output_dim, hidden_dim, dropout, num_layers)
    fun_predictor = Fun_Predictor(output_dim + extra_data_with_id.shape[1] - 1, hidden_dim)

    device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
    graph_nn.to(device)
    fun_predictor.to(device)

    optimizer_graph = torch.optim.Adam(graph_nn.parameters(), lr=0.001)
    optimizer_fun = torch.optim.Adam(fun_predictor.parameters(), lr=0.001)

    scheduler_graph = StepLR(optimizer_graph, step_size=7, gamma=0.1)
    scheduler_fun = StepLR(optimizer_fun, step_size=7, gamma=0.1)

    criterion = torch.nn.MSELoss()

    train_loader = DataLoader(molecule_data, batch_size=32, shuffle=True, collate_fn=collate_fn)

    epochs = 5
    for epoch in range(epochs):
        loss_graph = train_graph_nn(graph_nn, train_loader, optimizer_graph)
        loss_fun = train_fun_predictor(fun_predictor, graph_nn, molecule_data, extra_data_with_id, criterion, optimizer_fun)
        scheduler_graph.step()
        scheduler_fun.step()
        print(f'Epoch [{epoch+1}/{epochs}], Loss_Graph: {loss_graph:.4f}, Loss_Fun: {loss_fun:.4f}')


C:\Users\Elepikachu\anaconda3\lib\site-packages\torch\nn\modules\loss.py:535: UserWarning: Using a target size (torch.Size([1])) that is different to the input size (torch.Size([])). This will likely lead to incorrect results due to broadcasting. Please ensure they have the same size.
  return F.mse_loss(input, target, reduction=self.reduction)


Epoch [1/5], Loss_Graph: 0.2206, Loss_Fun: 0.2958
Epoch [2/5], Loss_Graph: 0.0500, Loss_Fun: 0.0827
Epoch [3/5], Loss_Graph: 0.0445, Loss_Fun: 0.0569
Epoch [4/5], Loss_Graph: 0.0463, Loss_Fun: 0.0509
Epoch [5/5], Loss_Graph: 0.0412, Loss_Fun: 0.0468


In [26]:
input_dim = 5  # 原子特征维度
output_dim = 1  # 图嵌入输出维度
hidden_dim = 4
dropout = 0.3
num_layers = 2

graph_nn = GraphNN(input_dim, output_dim, hidden_dim, dropout, num_layers)
x = torch.tensor([
    [0.1, 0.2, 0.3, 0.4, 0.5],  # Node 0 features
    [0.2, 0.3, 0.4, 0.5, 0.6],  # Node 1 features
    [0.3, 0.4, 0.5, 0.6, 0.7],  # Node 2 features
    [0.4, 0.5, 0.6, 0.7, 0.8]   # Node 3 features
], dtype=torch.float)

# 创建边索引，表示有向边 (source, target)
edge_index = torch.tensor([
    [0, 1, 2, 3],  # Source nodes of edges
    [1, 2, 3, 0]   # Target nodes of edges
], dtype=torch.long)

# 创建一个Batch对象模拟输入数据
class Batch:
    def __init__(self, x, edge_index, edge_attr=None):
        self.x = x
        self.edge_index = edge_index
        if edge_attr is not None:
            self.edge_attr = edge_attr

# 模拟没有边特征的情况
batch = Batch(x=x, edge_index=edge_index)
print(batch.x)
output = graph_nn(batch)
print(output.shape)

tensor([[0.1000, 0.2000, 0.3000, 0.4000, 0.5000],
        [0.2000, 0.3000, 0.4000, 0.5000, 0.6000],
        [0.3000, 0.4000, 0.5000, 0.6000, 0.7000],
        [0.4000, 0.5000, 0.6000, 0.7000, 0.8000]])
torch.Size([4, 1])


In [42]:
csv_file_molecules = 'molecules.csv'

molecule_data = load_data(csv_file_molecules)
for item in molecule_data:
    print(item['x'].shape)

torch.Size([7, 8])
torch.Size([7, 8])
torch.Size([7, 8])
torch.Size([7, 8])
torch.Size([7, 8])
torch.Size([17, 8])
torch.Size([91, 8])
torch.Size([91, 8])
torch.Size([91, 8])
torch.Size([91, 8])
torch.Size([95, 8])
torch.Size([95, 8])
torch.Size([95, 8])
torch.Size([95, 8])
torch.Size([95, 8])
torch.Size([95, 8])
torch.Size([95, 8])
torch.Size([95, 8])
torch.Size([95, 8])
torch.Size([95, 8])
torch.Size([95, 8])
torch.Size([95, 8])
torch.Size([95, 8])
torch.Size([95, 8])
torch.Size([95, 8])
torch.Size([95, 8])
torch.Size([95, 8])
torch.Size([95, 8])
torch.Size([95, 8])
torch.Size([95, 8])
torch.Size([95, 8])
torch.Size([95, 8])
torch.Size([95, 8])
torch.Size([95, 8])
torch.Size([95, 8])
torch.Size([95, 8])
torch.Size([95, 8])
torch.Size([95, 8])
torch.Size([95, 8])
torch.Size([95, 8])
torch.Size([95, 8])
torch.Size([95, 8])
torch.Size([95, 8])
torch.Size([95, 8])
torch.Size([95, 8])
torch.Size([95, 8])
torch.Size([95, 8])
torch.Size([95, 8])
torch.Size([95, 8])
torch.Size([95, 8])
torch

In [7]:
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch_geometric.data import Data, Batch
from torch_geometric.nn import GATConv
from torch.optim.lr_scheduler import StepLR
from torch.utils.data import DataLoader, random_split
from matplotlib import pyplot as plt
import numpy as np
import pandas as pd
from rdkit import Chem
from rdkit import DataStructs
from rdkit.Chem import AllChem, Draw, Lipinski, Descriptors
from rdkit.ML.Descriptors import MoleculeDescriptors
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler

def load_data(csv_file):
    data = pd.read_csv(csv_file, encoding='ANSI')
    molecules = []
    max_atoms = 158 #最大原子数
    max_edges = 0 # 最大边数
    for _, group in data.groupby('ID'):
        for _, row in group.iterrows():
            IE = row['IE']
            smiles = row['SMILES']
            molecule_id = row['ID']
            mol = Chem.MolFromSmiles(smiles)
            if mol is None:
                continue
            num_atoms = len(mol.GetAtoms())
            max_atoms = max(max_atoms, num_atoms)
            bonds = mol.GetBonds()
            num_edges = len(bonds)
            adj_matrix = Chem.GetAdjacencyMatrix(mol)
            atom_features = []
            for atom in mol.GetAtoms():
                features = [
                    atom.GetAtomicNum(),
                    atom.GetDegree(),
                    atom.GetFormalCharge(),
                    int(atom.HasProp('_ChiralityPossible')),
                    atom.GetTotalNumHs(),
                    atom.GetHybridization().real,
                    atom.GetExplicitValence(),
                    int(atom.GetIsAromatic()),
                ]
                atom_features.append(features)
            features = np.array(atom_features, dtype=np.float32)
            atom_features = torch.tensor(features, dtype=torch.float)
            if num_atoms < max_atoms:
                padding = torch.zeros((max_atoms - num_atoms, 8), dtype=torch.float)
                atom_features = torch.cat([atom_features, padding], dim=0)
            edge_index = torch.tensor(np.array(np.nonzero(adj_matrix)), dtype=torch.long)
            max_edges = max(max_edges, edge_index.shape[1])
            molecules.append({
                'x': atom_features,
                'edge_index': edge_index,
                'smiles': smiles,
                'id': molecule_id,
                'y': torch.tensor([IE], dtype=torch.float)
            })
    return molecules

def load_extra_data(csv_file):
    df = pd.read_csv(csv_file)
    id_list = df['ID'].values
    extra_data = df.drop(columns=['ID']).values
    scaler = StandardScaler()
    extra_data_scale = scaler.fit_transform(extra_data)
    extra_data_with_id = pd.DataFrame(extra_data_scale, columns=df.columns[1:])
    extra_data_with_id.insert(0, 'ID', id_list)
    return id_list, extra_data_with_id

class GraphNN(nn.Module):
    def __init__(self, input_dim, output_dim, hidden_dim=64, dropout=0.3, num_layers=2):
        super().__init__()
        self.encoder = nn.ModuleList([
            GATConv(input_dim if i == 0 else hidden_dim, hidden_dim)
            for i in range(num_layers)
        ])
        self.decoder = GATConv(hidden_dim, output_dim)
        self.dropout = dropout
        self.layer_norm = nn.LayerNorm(hidden_dim)
        self._initialize_weights()

    def _initialize_weights(self):
        for layer in self.encoder:
            if isinstance(layer, GATConv):
                self.initialize_gatconv_weights(layer)

        if isinstance(self.decoder, GATConv):
            self.initialize_gatconv_weights(self.decoder)

    def initialize_gatconv_weights(self, gat_layer):
        if gat_layer.lin_src is not None and gat_layer.lin_dst is not None:
            nn.init.xavier_uniform_(gat_layer.lin_src.weight)
            nn.init.xavier_uniform_(gat_layer.lin_dst.weight)
            if gat_layer.attn_vecs is not None:
                nn.init.xavier_uniform_(gat_layer.attn_vecs)

    def forward(self, batch):
        x, edge_index = batch.x, batch.edge_index
        for layer in self.encoder:
            x = F.dropout(x, p=self.dropout, training=self.training)
            x = layer(x, edge_index)
            x = F.relu(x)
            x = self.layer_norm(x)
        output = self.decoder(x, edge_index)
        graph_embeddings = global_mean_pool(output, batch.batch) 
        return graph_embeddings

def train_graph_nn(model, data_loader, optimizer, clip_val=1.0):
    model.train()
    total_loss = 0
    loss_fun = torch.nn.MSELoss()
    num_samples = 0
    for data in data_loader:
        optimizer.zero_grad()
        length = data.x.size(0)
        output = model(data.to(torch.device('cuda' if torch.cuda.is_available() else 'cpu')))
        #y = data.y.to(torch.device('cuda' if torch.cuda.is_available() else 'cpu'))
        #y = torch.zeros((length,1)).to(torch.device('cuda' if torch.cuda.is_available() else 'cpu'))
        y = data.y.to(torch.device('cuda' if torch.cuda.is_available() else 'cpu')).view(-1)
            
        # 检查形状是否匹配
        if output.size(0) != y.size(0):
            raise ValueError(f"Shape mismatch: output {output.shape}, y {y.shape}")
#         print(y.shape)
#         print(output.shape)
        loss = loss_fun(output.squeeze(), y)
        loss.backward()
        torch.nn.utils.clip_grad_norm_(model.parameters(), clip_val)
        optimizer.step()
        total_loss += loss.item() *  y.size(0)
        num_samples +=  y.size(0)
    avg_loss = total_loss / num_samples if num_samples > 0 else float('nan')
    return avg_loss

def collate_fn(batch):
    x_list = [data['x'] for data in batch]
    edge_index_list = [data['edge_index'] for data in batch]
    y_list = [data['y'] for data in batch]

    x = torch.cat(x_list, dim=0)
    edge_index = []
    num_nodes_accum = 0
    for i, edge_idx in enumerate(edge_index_list):
        num_nodes = x_list[i].size(0)  # 使用x的节点数而非边索引最大值
        edge_idx_offset = edge_idx + num_nodes_accum
        edge_index.append(edge_idx_offset)
        num_nodes_accum += num_nodes  # 累加实际节点数

    edge_index = torch.cat(edge_index, dim=1)
    y = torch.cat(y_list, dim=0)
    
    # Create batch tensor
    batch_indices = []
    for i, data in enumerate(batch):
        num_nodes = data['x'].size(0)
        batch_indices.append(torch.full((num_nodes,), i, dtype=torch.long))
    batch_tensor = torch.cat(batch_indices, dim=0)


    batch_data = Batch(x=x, edge_index=edge_index, y=y, batch=batch_tensor)
    return batch_data

class Fun_Predictor(nn.Module):
    def __init__(self, input_size, hidden_size, dropout_rate=0.5):
        super(Fun_Predictor, self).__init__()
        self.fc1 = nn.Linear(input_size, hidden_size)
       # self.bn1 = nn.BatchNorm1d(hidden_size)
        self.bn1 = nn.LayerNorm(hidden_size)
        self.dropout = nn.Dropout(dropout_rate)
        self.fc2 = nn.Linear(hidden_size, hidden_size)
        #self.bn2 = nn.BatchNorm1d(hidden_size)
        self.bn2 = nn.LayerNorm(hidden_size)
        self.fc3 = nn.Linear(hidden_size, hidden_size)
        self.fc4 = nn.Linear(hidden_size, 1)
        self._initialize_weights()

    def _initialize_weights(self):
        for m in self.modules():
            if isinstance(m, nn.Linear):
                nn.init.xavier_uniform_(m.weight)
                if m.bias is not None:
                    nn.init.constant_(m.bias, 0)

    def forward(self, combined_features):
        x = torch.relu(self.fc1(combined_features))
        x = self.bn1(x)
        x = self.dropout(x)
        x = torch.relu(self.fc2(x))
        x = self.bn2(x)
        x = self.dropout(x)
        x = torch.relu(self.fc3(x))
        predict_IE = self.fc4(x)
        return predict_IE

def train_fun_predictor(model_fun, graph_nn, molecule_data, extra_data_with_id, criterion, optimizer_fun):
    model_fun.train()
    graph_nn.eval()
    total_loss = 0
    
    # 创建一个字典，将额外特征与id匹配，去除id列
    extra_data_dict = {row['ID']: torch.tensor(row[1:], dtype=torch.float32) for _, row in extra_data_with_id.iterrows()}
    
    # 初始化一个列表保存额外特征值
    extra_features = []
    targets = []
    graph_embeddings = []

    # 遍历所有额外特征数据，并为每个分子获取对应的图嵌入和特征
    for _, row in extra_data_with_id.iterrows():
        id_ = row['ID']
        extra_feature_tensor = extra_data_dict[id_].unsqueeze(0)
        molecule_data_for_id = next((data for data in molecule_data if data['id'] == id_), None)
        if molecule_data_for_id is None:
            raise ValueError(f"No mol data found for ID: {id_}")
        x = molecule_data_for_id['x'] # 原子特征
        edge_index = molecule_data_for_id['edge_index'] # 邻接矩阵
        graph_data = Data(x=x, edge_index=edge_index)
        
        # 获取图嵌入
        with torch.no_grad():
            embedding = graph_nn(graph_data.to(torch.device('cuda' if torch.cuda.is_available() else 'cpu')))
            pooled_embedding = torch.mean(embedding, dim=0).unsqueeze(0) #平均池化
        
        # 拼接图嵌入和额外特征
        combined_features = torch.cat((pooled_embedding, extra_feature_tensor), dim=1)
        extra_features.append(combined_features)
        targets.append(molecule_data_for_id['y'])
        graph_embeddings.append(pooled_embedding)

    # 将所有特征和目标转换为张量
    extra_features = torch.cat(extra_features, dim=0)
    targets = torch.cat(targets, dim=0)
    graph_embeddings = torch.cat(graph_embeddings, dim=0)

    # 训练循环
    for i in range(len(extra_features)):
        combined_feature = extra_features[i].unsqueeze(0)
        target = targets[i].unsqueeze(0)
        optimizer_fun.zero_grad()
        prediction = model_fun(combined_feature.to(torch.device('cuda' if torch.cuda.is_available() else 'cpu')))
        loss = criterion(prediction.squeeze(), target.to(torch.device('cuda' if torch.cuda.is_available() else 'cpu')))
        loss.backward()
        optimizer_fun.step()
        total_loss += loss.item()

    avg_loss = total_loss / len(extra_data_with_id)
    return avg_loss

# 示例使用
if __name__ == "__main__":
    csv_file_molecules = 'molecules.csv'
    csv_file_extra = 'extra_data.csv'

    molecule_data = load_data(csv_file_molecules)
    id_list, extra_data_with_id = load_extra_data(csv_file_extra)
    
    # 分割数据集为训练集、验证集和测试集
    train_ratio = 0.7
    val_ratio = 0.15
    test_ratio = 0.15

    dataset_size = len(molecule_data)
    train_size = int(train_ratio * dataset_size)
    val_size = int(val_ratio * dataset_size)
    test_size = dataset_size - train_size - val_size

    train_dataset, val_dataset, test_dataset = random_split(molecule_data, [train_size, val_size, test_size])

    input_dim = 8  # 原子特征维度
    output_dim = 1  # 图嵌入输出维度
    hidden_dim = 64
    dropout = 0.3
    num_layers = 2

    graph_nn = GraphNN(input_dim, output_dim, hidden_dim, dropout, num_layers)
    fun_predictor = Fun_Predictor(output_dim + extra_data_with_id.shape[1] - 1, hidden_dim)

    device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
    graph_nn.to(device)
    fun_predictor.to(device)

    optimizer_graph = torch.optim.Adam(graph_nn.parameters(), lr=0.001)
    optimizer_fun = torch.optim.Adam(fun_predictor.parameters(), lr=0.001)

    scheduler_graph = StepLR(optimizer_graph, step_size=7, gamma=0.1)
    scheduler_fun = StepLR(optimizer_fun, step_size=7, gamma=0.1)

    criterion = torch.nn.MSELoss()

    train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True, collate_fn=collate_fn)
    val_loader = DataLoader(val_dataset, batch_size=32, shuffle=False, collate_fn=collate_fn)

    epochs = 5
    for epoch in range(epochs):
        loss_graph = train_graph_nn(graph_nn, train_loader, optimizer_graph)
        loss_fun = train_fun_predictor(fun_predictor, graph_nn, molecule_data, extra_data_with_id, criterion, optimizer_fun)
        scheduler_graph.step()
        scheduler_fun.step()
        print(f'Epoch [{epoch+1}/{epochs}], Loss_Graph: {loss_graph:.4f}, Loss_Fun: {loss_fun:.4f}')
        
        # 验证步骤
        graph_nn.eval()
        fun_predictor.eval()
        val_loss = 0
        for data in val_loader:
            with torch.no_grad():
                length = data.x.size(0)
                output = graph_nn(data.to(device)).squeeze()
                y = torch.zeros((length,1)).to(device)
                val_loss += criterion(output, y).item() * len(y)
        val_avg_loss = val_loss / len(val_dataset)
        print(f'Validation Loss: {val_avg_loss:.4f}')

        # 保存最佳模型
        if val_avg_loss < best_val_loss:
            best_val_loss = val_avg_loss
            torch.save(graph_nn.state_dict(), 'best_graph_nn.pth')
            torch.save(fun_predictor.state_dict(), 'best_fun_predictor.pth')


C:\Users\Elepikachu\anaconda3\lib\site-packages\torch\nn\modules\loss.py:535: UserWarning: Using a target size (torch.Size([1])) that is different to the input size (torch.Size([])). This will likely lead to incorrect results due to broadcasting. Please ensure they have the same size.
  return F.mse_loss(input, target, reduction=self.reduction)


Epoch [1/5], Loss_Graph: 0.4542, Loss_Fun: 0.2484
Validation Loss: 133.4955


C:\Users\Elepikachu\anaconda3\lib\site-packages\torch\nn\modules\loss.py:535: UserWarning: Using a target size (torch.Size([5056, 1])) that is different to the input size (torch.Size([32])). This will likely lead to incorrect results due to broadcasting. Please ensure they have the same size.
  return F.mse_loss(input, target, reduction=self.reduction)
C:\Users\Elepikachu\anaconda3\lib\site-packages\torch\nn\modules\loss.py:535: UserWarning: Using a target size (torch.Size([4424, 1])) that is different to the input size (torch.Size([28])). This will likely lead to incorrect results due to broadcasting. Please ensure they have the same size.
  return F.mse_loss(input, target, reduction=self.reduction)


NameError: name 'best_val_loss' is not defined

In [ ]:
# 加载最佳模型进行测试
graph_nn.load_state_dict(torch.load('best_graph_nn.pth'))
fun_predictor.load_state_dict(torch.load('best_fun_predictor.pth'))

test_fun_predictor(fun_predictor, graph_nn, test_dataset, extra_data_with_id, criterion)

In [91]:
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch_geometric.data import Data, Batch
from torch_geometric.nn import GATConv
from torch.optim.lr_scheduler import StepLR
from torch.utils.data import DataLoader, random_split
from matplotlib import pyplot as plt
import numpy as np
import pandas as pd
from rdkit import Chem
from rdkit import DataStructs
from rdkit.Chem import AllChem, Draw, Lipinski, Descriptors
from rdkit.ML.Descriptors import MoleculeDescriptors
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler

def load_data(csv_file):
    data = pd.read_csv(csv_file, encoding='ANSI')
    molecules = []
    max_atoms = 158 #最大原子数
    max_edges = 0 # 最大边数
    for _, group in data.groupby('ID'):
        for _, row in group.iterrows():
            IE = row['IE']
            smiles = row['SMILES']
            molecule_id = row['ID']
            mol = Chem.MolFromSmiles(smiles)
            if mol is None:
                continue
            num_atoms = len(mol.GetAtoms())
            max_atoms = max(max_atoms, num_atoms)
            bonds = mol.GetBonds()
            num_edges = len(bonds)
            adj_matrix = Chem.GetAdjacencyMatrix(mol)
            atom_features = []
            for atom in mol.GetAtoms():
                features = [
                    atom.GetAtomicNum(),
                    atom.GetDegree(),
                    atom.GetFormalCharge(),
                    int(atom.HasProp('_ChiralityPossible')),
                    atom.GetTotalNumHs(),
                    atom.GetHybridization().real,
                    atom.GetExplicitValence(),
                    int(atom.GetIsAromatic()),
                ]
                atom_features.append(features)
            features = np.array(atom_features, dtype=np.float32)
            atom_features = torch.tensor(features, dtype=torch.float)
            if num_atoms < max_atoms:
                padding = torch.zeros((max_atoms - num_atoms, 8), dtype=torch.float)
                atom_features = torch.cat([atom_features, padding], dim=0)
            edge_index = torch.tensor(np.array(np.nonzero(adj_matrix)), dtype=torch.long)
            max_edges = max(max_edges, edge_index.shape[1])
            molecules.append({
                'x': atom_features,
                'edge_index': edge_index,
                'smiles': smiles,
                'id': molecule_id,
                'y': torch.tensor([IE], dtype=torch.float)
            })
    return molecules

def load_extra_data(csv_file):
    df = pd.read_csv(csv_file)
    id_list = df['ID'].values
    extra_data = df.drop(columns=['ID','IE','SMILES']).values
    scaler = StandardScaler()
    extra_data_scale = scaler.fit_transform(extra_data)
    print(extra_data_scale)
    extra_data_with_id = pd.DataFrame(extra_data_scale, columns=df.columns[1:])
    extra_data_with_id.insert(0, 'ID', id_list)
    return id_list, extra_data_with_id

class GraphNN(nn.Module):
    def __init__(self, input_dim, output_dim, hidden_dim=64, dropout=0.3, num_layers=2):
        super().__init__()
        self.encoder = nn.ModuleList([
            GATConv(input_dim if i == 0 else hidden_dim, hidden_dim)
            for i in range(num_layers)
        ])
        self.decoder = GATConv(hidden_dim, output_dim)
        self.dropout = dropout
        self.layer_norm = nn.LayerNorm(hidden_dim)
        self._initialize_weights()

    def _initialize_weights(self):
        for layer in self.encoder:
            if isinstance(layer, GATConv):
                self.initialize_gatconv_weights(layer)

        if isinstance(self.decoder, GATConv):
            self.initialize_gatconv_weights(self.decoder)

    def initialize_gatconv_weights(self, gat_layer):
        if gat_layer.lin_src is not None and gat_layer.lin_dst is not None:
            nn.init.xavier_uniform_(gat_layer.lin_src.weight)
            nn.init.xavier_uniform_(gat_layer.lin_dst.weight)
            if gat_layer.attn_vecs is not None:
                nn.init.xavier_uniform_(gat_layer.attn_vecs)

    def forward(self, batch):
        x, edge_index = batch.x, batch.edge_index
        for layer in self.encoder:
            x = F.dropout(x, p=self.dropout, training=self.training)
            x = layer(x, edge_index)
            x = F.relu(x)
            x = self.layer_norm(x)
        output = self.decoder(x, edge_index)
        return output

def train_graph_nn(model, data_loader, optimizer, clip_val=1.0):
    model.train()
    total_loss = 0
    loss_fun = torch.nn.MSELoss()
    num_samples = 0
    for data in data_loader:
        optimizer.zero_grad()
        length = data.x.size(0)
        output = model(data.to(torch.device('cuda' if torch.cuda.is_available() else 'cpu')))
        #y = data.y.to(torch.device('cuda' if torch.cuda.is_available() else 'cpu'))
        y = torch.zeros((length,1)).to(torch.device('cuda' if torch.cuda.is_available() else 'cpu'))
#         print(y.shape)
#         print(output.shape)
        loss = loss_fun(output.squeeze(), y)
        loss.backward()
        torch.nn.utils.clip_grad_norm_(model.parameters(), clip_val)
        optimizer.step()
        total_loss += loss.item() * len(y)
        num_samples += len(y)
    avg_loss = total_loss / num_samples if num_samples > 0 else float('nan')
    return avg_loss

def collate_fn(batch):
    x_list = [data['x'] for data in batch]
    edge_index_list = [data['edge_index'] for data in batch]
    y_list = [data['y'] for data in batch]

    x = torch.cat(x_list, dim=0)
    edge_index = []
    num_nodes_accum = 0
    for edge_idx in edge_index_list:
        edge_idx_offset = edge_idx + num_nodes_accum
        edge_index.append(edge_idx_offset)
        num_nodes_accum += edge_idx.max().item() + 1

    edge_index = torch.cat(edge_index, dim=1)
    y = torch.cat(y_list, dim=0)

    batch_data = Batch(x=x, edge_index=edge_index, y=y)
    return batch_data

class Fun_Predictor(nn.Module):
    def __init__(self, input_size, hidden_size, dropout_rate=0.5):
        super(Fun_Predictor, self).__init__()
        self.fc1 = nn.Linear(input_size, hidden_size)
       # self.bn1 = nn.BatchNorm1d(hidden_size)
        self.bn1 = nn.LayerNorm(hidden_size)
        self.dropout = nn.Dropout(dropout_rate)
        self.fc2 = nn.Linear(hidden_size, hidden_size)
        #self.bn2 = nn.BatchNorm1d(hidden_size)
        self.bn2 = nn.LayerNorm(hidden_size)
        self.fc3 = nn.Linear(hidden_size, hidden_size)
        self.fc4 = nn.Linear(hidden_size, 1)
        self._initialize_weights()

    def _initialize_weights(self):
        for m in self.modules():
            if isinstance(m, nn.Linear):
                nn.init.xavier_uniform_(m.weight)
                if m.bias is not None:
                    nn.init.constant_(m.bias, 0)

    def forward(self, combined_features):
        x = torch.relu(self.fc1(combined_features))
        x = self.bn1(x)
        x = self.dropout(x)
        x = torch.relu(self.fc2(x))
        x = self.bn2(x)
        x = self.dropout(x)
        x = torch.relu(self.fc3(x))
        predict_IE = self.fc4(x)
        return predict_IE

def train_fun_predictor(model_fun, graph_nn, molecule_data, extra_data_with_id, criterion, optimizer_fun):
    model_fun.train()
    graph_nn.eval()
    total_loss = 0
    
    # 创建一个字典，将额外特征与id匹配，去除id列
    extra_data_dict = {row['ID']: torch.tensor(row[1:], dtype=torch.float32) for _, row in extra_data_with_id.iterrows()}
    
    # 初始化一个列表保存额外特征值
    extra_features = []
    targets = []
    graph_embeddings = []

    # 遍历所有额外特征数据，并为每个分子获取对应的图嵌入和特征
    for _, row in extra_data_with_id.iterrows():
        id_ = row['ID']
        extra_feature_tensor = extra_data_dict[id_].unsqueeze(0)
        molecule_data_for_id = next((data for data in molecule_data if data['id'] == id_), None)
        if molecule_data_for_id is None:
            raise ValueError(f"No mol data found for ID: {id_}")
        x = molecule_data_for_id['x'] # 原子特征
        edge_index = molecule_data_for_id['edge_index'] # 邻接矩阵
        graph_data = Data(x=x, edge_index=edge_index)
        
        # 获取图嵌入
        with torch.no_grad():
            embedding = graph_nn(graph_data.to(torch.device('cuda' if torch.cuda.is_available() else 'cpu')))
            pooled_embedding = torch.mean(embedding, dim=0).unsqueeze(0) #平均池化
    #    print(pooled_embedding, extra_feature_tensor)
        # 拼接图嵌入和额外特征
        combined_features = torch.cat((pooled_embedding, extra_feature_tensor), dim=1)
        extra_features.append(combined_features)
        targets.append(molecule_data_for_id['y'])
        graph_embeddings.append(pooled_embedding)

    # 将所有特征和目标转换为张量
    extra_features = torch.cat(extra_features, dim=0)
    targets = torch.cat(targets, dim=0)
    graph_embeddings = torch.cat(graph_embeddings, dim=0)

    # 训练循环
    for i in range(len(extra_features)):
        combined_feature = extra_features[i].unsqueeze(0)
        target = targets[i].unsqueeze(0)
        optimizer_fun.zero_grad()
        prediction = model_fun(combined_feature.to(torch.device('cuda' if torch.cuda.is_available() else 'cpu')))
        loss = criterion(prediction.squeeze(), target.to(torch.device('cuda' if torch.cuda.is_available() else 'cpu')))
        loss.backward()
        optimizer_fun.step()
        total_loss += loss.item()
    
    avg_loss = total_loss / len(extra_data_with_id)
    return avg_loss

# 示例使用
if __name__ == "__main__":
    csv_file_molecules = 'molecules.csv'
    csv_file_extra = 'extra_data.csv'
    csv_file = 'solution_properties_converted.csv'

    molecule_data = load_data(csv_file)
    id_list, extra_data_with_id = load_extra_data(csv_file)
    
    train_ratio = 0.85
    val_ratio = 0.15

    dataset_size = len(molecule_data)
    train_size = int(train_ratio * dataset_size)
    test_size = dataset_size - train_size

    train_dataset, test_dataset = random_split(molecule_data, [train_size, test_size])

    input_dim = 8  # 原子特征维度
    output_dim = 1  # 图嵌入输出维度
    hidden_dim = 64
    dropout = 0.3
    num_layers = 2

    graph_nn = GraphNN(input_dim, output_dim, hidden_dim, dropout, num_layers)
    fun_predictor = Fun_Predictor(output_dim + extra_data_with_id.shape[1] - 1, hidden_dim)

    device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
    graph_nn.to(device)
    fun_predictor.to(device)

    optimizer_graph = torch.optim.Adam(graph_nn.parameters(), lr=0.001)
    optimizer_fun = torch.optim.Adam(fun_predictor.parameters(), lr=0.001)

    scheduler_graph = StepLR(optimizer_graph, step_size=7, gamma=0.1)
    scheduler_fun = StepLR(optimizer_fun, step_size=7, gamma=0.1)

    criterion = torch.nn.MSELoss()

    train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True, collate_fn=collate_fn)

    epochs = 5
    for epoch in range(epochs):
        loss_graph = train_graph_nn(graph_nn, train_loader, optimizer_graph)
        loss_fun = train_fun_predictor(fun_predictor, graph_nn, molecule_data, extra_data_with_id, criterion, optimizer_fun)
        scheduler_graph.step()
        scheduler_fun.step()
        print(f'Epoch [{epoch+1}/{epochs}], Loss_Graph: {loss_graph:.4f}, Loss_Fun: {loss_fun:.4f}')

[[-0.8254219  -0.30208242 -0.03985613]
 [ 0.55363249 -0.30208242 -0.03985613]
 [ 2.27745049 -0.30208242 -0.03985613]
 ...
 [ 0.03648709 -0.30208242 -0.03985479]
 [ 0.03648709  0.20967644 -0.03985479]
 [ 0.03648709  0.20967644 -0.03985479]]


ValueError: Shape of passed values is (1682, 3), indices imply (1682, 5)

In [95]:
def test_fun_predictor(model_fun, graph_nn, molecule_data, extra_data_with_id, criterion):
    model_fun.eval()
    graph_nn.eval()
    total_loss = 0
    all_predictions = []
    all_targets = []
    
    extra_data_dict = {row['ID']: torch.tensor(row[1:], dtype=torch.float32) for _, row in extra_data_with_id.iterrows()}
    
    extra_features = []
    targets = []
    graph_embeddings = []

    for data in molecule_data:
        id_ = data['id']
        extra_feature_tensor = extra_data_dict.get(id_).unsqueeze(0)
        if extra_feature_tensor is None:
            raise ValueError(f"No extra data found for ID: {id_}")
        x = data['x'] 
        edge_index = data['edge_index'] 
        graph_data = Data(x=x, edge_index=edge_index)
        
        with torch.no_grad():
            embedding = graph_nn(graph_data.to(torch.device('cuda' if torch.cuda.is_available() else 'cpu')))
            pooled_embedding = torch.mean(embedding, dim=0).unsqueeze(0) 
     #   print(pooled_embedding, extra_feature_tensor)
        
        combined_features = torch.cat((pooled_embedding, extra_feature_tensor), dim=1)
        extra_features.append(combined_features)
        targets.append(data['y'])
        graph_embeddings.append(pooled_embedding)

    extra_features = torch.cat(extra_features, dim=0)
    targets = torch.cat(targets, dim=0)
    graph_embeddings = torch.cat(graph_embeddings, dim=0)

    for i in range(len(extra_features)):
        combined_feature = extra_features[i].unsqueeze(0)
        target = targets[i].unsqueeze(0)
        with torch.no_grad():
            prediction = model_fun(combined_feature.to(torch.device('cuda' if torch.cuda.is_available() else 'cpu')))
            loss = criterion(prediction.squeeze(), target.to(torch.device('cuda' if torch.cuda.is_available() else 'cpu')))
            total_loss += loss.item()
            all_predictions.append(prediction.cpu().numpy())
            all_targets.append(target.cpu().numpy())

    avg_loss = total_loss / len(extra_data_with_id)
    all_predictions = np.concatenate(all_predictions)
    all_targets = np.concatenate(all_targets)
    mse = mean_squared_error(all_targets, all_predictions)
    r2 = r2_score(all_targets, all_predictions)
    print(f'Test Loss: {avg_loss:.4f}, MSE: {mse:.4f}, R^2: {r2:.4f}')
    return avg_loss, mse, r2

test_fun_predictor(fun_predictor, graph_nn, train_dataset, extra_data_with_id, criterion)

C:\Users\Elepikachu\anaconda3\lib\site-packages\torch\nn\modules\loss.py:535: UserWarning: Using a target size (torch.Size([1])) that is different to the input size (torch.Size([])). This will likely lead to incorrect results due to broadcasting. Please ensure they have the same size.
  return F.mse_loss(input, target, reduction=self.reduction)


Test Loss: 0.0406, MSE: 0.0478, R^2: -0.2609


(0.04063962779740436, 0.047834747, -0.2609063136995706)

In [83]:
print(len(extra_data_with_id))

1682


In [28]:
Chem.rdFingerprintGenerator

<module 'rdkit.Chem.rdFingerprintGenerator' from 'C:\\Users\\Elepikachu\\anaconda3\\lib\\site-packages\\rdkit\\Chem\\rdFingerprintGenerator.pyd'>